# Scraping SNAP decision data from USDA.gov

I used Winny de Jong's super accessible <a href="https://datajournalism.com/watch/python-for-journalists">Python for Journalists</a> course on  datajournalism.com as my introduction to web scraping. From there, I taught myself to iterate across multiple pages.  Finally, I pair coded with a back-end engineer friend to learn other ways of achieving the same goals (storing data in a dictionary instead of a list; iterating through URLs; etc). Thanks, Winny and Nashaad! 

I decided to practice by scraping the USDA's list of SNAP retailers that have been blacklisted or OKed after complaints. When I started this project, my local corner store had recently started offering more groceries. But the owner told me could not accept SNAP because a former business partner had committed fraud. That's too bad, because great, my neighborhood had lots of EBT users, but not many grocery stores. SNAP blacklist decisions are final; I was curious how often similar situations impacted neighborhoods like mine. 

Steps:
1. Scrape and clean the SNAP decision data **(this notebook)**.
2. Compare the data against a map of food deserts and test my hypotheses: 
    - There are more blacklisted SNAP retailers in food deserts than in more-resourced areas
    - If SNAP could be accepted at currently blacklisted locations, some food deserts would noticeably shrink.
4. Other directions to explore: 
    - Which partners have changed ownership since their decisions? 
    - Is there anything in the full text of the agency decisions worth looking into?


## Approach #1

Winny's approach is nice because it is simple; I summarized it below. The main drawback is that this way only scrapes one URL; I need to scrape 340.

In [28]:
# import libraries
# ----------------
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd

# prep csv file
# -------------
# create a csv file to store data from the page
f = open('SnapDecisions.csv', 'w', encoding='utf8', newline='') # (w = writeable)
# create a writer to write data to the csv
writer = csv.writer(f, delimiter=',') 
# write the header row
writer.writerow(['title', 'address', 'decision', 'decisionDate'])


# request and view page data
# --------------------------
# request the page; check that it loaded. Flashbacks to my website admin days! 
url = 'https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=1'
page = requests.get(url) #use the get function/method from the requests library to store the url in an object called page
# page
# grab the page HTML using Beautiful Soup; looks like a simple Drupal page w/ HTML table
soup = BeautifulSoup(page.content, 'html.parser')
# soup
# find the table; this page just has one
table = soup.find('table')
# table


"""
# explore data
# ------------
# find all the rows in the table; store in an object called "rows"
row = table.find_all('tr') # (table. or soup. both work since there is only one table)
rows
# if needed, find a specific row by row number (header row = 0)
rows[1]
# find all the cells in a given row (<th> = table header; <td> = table data)
cells = table.find_all('td')
cells = row[1].find_all('td')
# find a specific cell; use .text to view text without HTML
cells[0].text

"""

# organize data
# -------------
# create a for loop to store table data in a list. 
# 1. find each cell in a row; store in a variable called 'cells' 
# 2. the position of the cell determines it's column; create a header for each column
# 4. create a list of headers called rowData 
# 5. for each row in the table, create the rowData list and write to the csv
for row in table.find_all('tr'):
        cells = row.find_all(['th','td']) 
        title = cells[0].text
        address = cells[1].text
        decision = cells[2].text
        decisionDate = cells[3].text
        rowData = [title, address, decision, decisionDate]
#         print(rowData)
#         write to csv
#         ------------
        writer.writerow(rowData)

    
# check that it worked
# --------------------
# use the bash command "ls" to confirm that a file was created.
# ls
# confirm that the file contains data. Either check manually, or use a Pandas dataframe:
df = pd.read_csv('SnapDecisions.csv')
# df.shape # right amount of data?
df.head(5) # data looks right?




,title,address,decision,decisionDate
0,Title,Address,FAD,FAD Date
1,Bab El Salam Restaurant LLC,"129 W. Centre Ave. \nPortage, MI 49024-5335 ...",Denial,10/26/2016
2,LK Mini Market,"6025 Pacific Ave. \nStockton, CA 95207",Denial,10/26/2016
3,Euclid Market,"4268 Euclid Ave. \nSan Diego, CA 92115-4977 ...",Denial,10/12/2016
4,Bali’s Food Mart,"12259 Veterans Memorial Drive \nHouston, TX 7...",Denial,10/25/2016


# Approach #2
Big thanks to my friend Nashaad, who helped me explore other ways to do this! 

New best practices I learned:
- Use "f" in Python to format logically similar URLs, add them to a variable, and iterate through to avoid adding them manually like I did earlier.
- Store row data in a dictionary instead of a list to give each cell an explicit key. This provides more context and gives the flexibility to view data with json instead of pandas, and to use json instead of a csv later. 
- Use "print" within a loop to track what the scraper is doing and throw errors.
- Use short, random sleep steps instead of long, consistent ones to throttle without slowing your scraper down.

Othr useful things I learned:
- "print(TK.prettify())" - prints nicely formatted HTML.
- You can use .find('a').get('b') together to find an constant and get a related variable.
- "with open" opens AND closes your csv for you.
- In Python, use "=" to define a variable or object; "def" to define a function.
- Like in Excel, it is easy to replace or strip text in a string.

In [58]:
# import libraries
# ----------------
import csv
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import pandas as pd
import json


# define a function to parse table data, then write it to a csv
# -------------------------------------------------------------
def parse_and_csv(html):
    
    # request page data
    # -----------------
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    """
    # make soup object for the first row of the csv--the header
    header = table.find('tr')
    headings = header.find_all('th')
    headings_list = []
    for th in headings:
        headings_list.append(th.text)
    """

    # make soup objects for the other rows in the csv--the data
    body = table.find('tbody')
    rows = body.find_all('tr')     

    
    # parse table data using a dictionary
    # -----------------------------------
    # store row data in a list
    rows_list = []
    for row in rows:
        tds = row.find_all('td')
        cells_dict = {
            # strip white space from text
            'Title': tds[0].text.strip(),
            # strip whitespace and line breaks from address field
            'Address': tds[1].text.replace('  \n', ', ').strip(),
            'Decision': tds[2].text.strip(),
            'Decision Date': tds[3].text.strip()
        }

        # make a list of dictionaries
        rows_list.append(cells_dict)
    
    
    # view dictionary using json
    # --------------------------
    # print(json.dumps(rows_list, indent=4))


    # write dictionary to the csv
    # ---------------------------
    with open('SNAPple.csv', mode='a') as event_file:
        #use the DictWriter argument to write dictionaries to a csv
        writer = csv.DictWriter(event_file, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        # iterate through all the row_list items
        # and write row to csv
        for item in rows_list:
            writer.writerow(item)
            

    # check that it worked
    # --------------------        
    df = pd.read_csv('SNAPple.csv')
    df


# before running the function above, write the header row to the csv
# ------------------------------------------------------------------
with open('SNAPple.csv', mode='w') as event_file:
    # make a list of names for the header
    fieldnames = ['Title', 'Address', 'Decision', 'Decision Date']
    writer = csv.DictWriter(event_file, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    # write the csv header (once)
    writer.writeheader() 
    
    
# now that the csv has a header, run the 'parse_and_csv' function for all pages
# -----------------------------------------------------------------------------
for i in range(340):
    # use f to format strings in python3
    url = f'https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page={i}'
    print(f'Currently scraping url: [{url}]')    # add a status message
    solong = random.randrange(0, 25)/100    # add a sleep step
    print(f'sleeping for {solong}')    # add a status message
    sleep(solong)
    
    # for i in range(): url = f'https...{i} pulls all urls into the url variable.
    # now, we can use "requests.get" in one fell swoop on all the urls.
    # this is where I was stuck earlier!
    page = requests.get(url)
    
    # if each page loads, run the "parse_and_csv" function defined above
    if page.status_code == 200:
        parse_and_csv(page.text)
    # if the page doesn't load, throw an error message
    else: 
        print(f'Cry for help from url {url}: Yikes, I am breaking!') 
        break

Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=0]
sleeping for 0.05
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=1]
sleeping for 0.05
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=2]
sleeping for 0.24
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=3]
sleeping for 0.13
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=4]
sleeping for 0.2
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=5]
sleeping for 0.02
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=6]
sleeping for 0.23
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=7]
sleeping for 0.2
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&

Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=70]
sleeping for 0.09
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=71]
sleeping for 0.05
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=72]
sleeping for 0.22
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=73]
sleeping for 0.06
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=74]
sleeping for 0.2
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=75]
sleeping for 0.22
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=76]
sleeping for 0.14
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=77]
sleeping for 0.11
Currently scraping url: [https://www.fns.usda.gov/snap/fa

Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=140]
sleeping for 0.11
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=141]
sleeping for 0.09
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=142]
sleeping for 0.16
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=143]
sleeping for 0.14
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=144]
sleeping for 0.21
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=145]
sleeping for 0.04
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=146]
sleeping for 0.12
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=147]
sleeping for 0.04
Currently scraping url: [https://www.fns.usda.go

Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=209]
sleeping for 0.03
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=210]
sleeping for 0.1
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=211]
sleeping for 0.22
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=212]
sleeping for 0.08
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=213]
sleeping for 0.2
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=214]
sleeping for 0.22
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=215]
sleeping for 0.05
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=216]
sleeping for 0.14
Currently scraping url: [https://www.fns.usda.gov/

Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=278]
sleeping for 0.17
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=279]
sleeping for 0.08
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=280]
sleeping for 0.12
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=281]
sleeping for 0.13
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=282]
sleeping for 0.03
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=283]
sleeping for 0.13
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=284]
sleeping for 0.18
Currently scraping url: [https://www.fns.usda.gov/snap/fad?title=&field_fiscal_year_value=&page=285]
sleeping for 0.05
Currently scraping url: [https://www.fns.usda.go